<a href="https://colab.research.google.com/github/anubhavanand1516/Stylumia_assignment/blob/main/kaggle_competitions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q opendatasets

import opendatasets as od
import pandas as pd

od.download('https://www.kaggle.com/competitions/extracting-attributes-from-fashion-images-jan-2024/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: anubhavanand1516
Your Kaggle Key: ··········


100%|██████████| 508M/508M [00:17<00:00, 30.8MB/s]



Extracting archive ./extracting-attributes-from-fashion-images-jan-2024/extracting-attributes-from-fashion-images-jan-2024.zip to ./extracting-attributes-from-fashion-images-jan-2024


In [ ]:
import pandas as pd
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
labels_df = pd.read_csv('/content/extracting-attributes-from-fashion-images-jan-2024/train.csv')

In [ ]:
train_data_dir = '/content/extracting-attributes-from-fashion-images-jan-2024/train'
test_data_dir = '/content/extracting-attributes-from-fashion-images-jan-2024/test'
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
)


In [ ]:
labels_df['label'] = labels_df['label'].astype(str)
batch_size = 64
img_height = 112
img_width = 112


In [ ]:
train_generator = datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory=train_data_dir,
    x_col='file_name',  # Column containing the image filenames
    y_col='label',  # Column containing the labels
    color_mode = "rgb",
    subset='training',
    batch_size=batch_size,
    target_size=(img_height, img_width),
    class_mode='categorical'
)

Found 14713 validated image filenames belonging to 7 classes.


In [ ]:
validation_generator = datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory=train_data_dir,
    x_col='file_name',
    y_col='label',
    subset='validation',
    batch_size=batch_size,
    target_size=(img_height, img_width),
    class_mode='categorical'
)

Found 3678 validated image filenames belonging to 7 classes.


In [ ]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 20
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/20
230/230 [==============================] - 1167s 5s/step - loss: 1.6540 - accuracy: 0.3596 - val_loss: 1.8052 - val_accuracy: 0.3165
Epoch 2/20
230/230 [==============================] - 1093s 5s/step - loss: 1.3548 - accuracy: 0.4767 - val_loss: 1.6538 - val_accuracy: 0.3464
Epoch 3/20
230/230 [==============================] - 1085s 5s/step - loss: 1.2070 - accuracy: 0.5372 - val_loss: 1.5772 - val_accuracy: 0.3475
Epoch 4/20
230/230 [==============================] - 1092s 5s/step - loss: 1.1155 - accuracy: 0.5731 - val_loss: 1.5578 - val_accuracy: 0.3864
Epoch 5/20
230/230 [==============================] - 1081s 5s/step - loss: 1.0390 - accuracy: 0.5960 - val_loss: 1.5341 - val_accuracy: 0.3883
Epoch 6/20
230/230 [==============================] - 1088s 5s/step - loss: 0.9673 - accuracy: 0.6243 - val_loss: 1.5793 - val_accuracy: 0.3662
Epoch 7/20
230/230 [==============================] - 1084s 5s/step - loss: 0.9024 - accuracy: 0.6422 - val_loss: 1.7295 - val_accuracy:

In [ ]:
model.save('trouser_fit_type_model.h5')

In [ ]:
sample_submission = pd.read_csv('/content/extracting-attributes-from-fashion-images-jan-2024/sample_submission.csv')


In [ ]:

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_generator = test_datagen.flow_from_dataframe(
    sample_submission,
    directory='/content/extracting-attributes-from-fashion-images-jan-2024/test',
    x_col='file_name',
    y_col=None,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False
)

Found 5751 validated image filenames.


In [ ]:
test_predictions = model.predict(test_generator)
predicted_labels = test_predictions.argmax(axis=1)

90/90 [==============================] - 135s 1s/step


In [ ]:
test_filenames = test_generator.filenames

In [ ]:
submission_data = {'file_name': test_filenames, 'label': predicted_labels}
submission_df = pd.DataFrame(submission_data)


In [ ]:
submission_df.to_csv('submission.csv', index=False)